In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from catboost import cv, Pool
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
RS = 121212
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE

In [2]:
sub = pd.read_csv('submission.csv', sep=';')

In [3]:
sub

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3637,3637,0
3638,3638,0
3639,3639,0
3640,3640,0


In [4]:
data_test = pd.read_csv('test.csv', sep=';', low_memory=False, parse_dates=['report_date'])

data_test

data_test = pd.read_csv('test.csv',  low_memory=False, parse_dates=['report_date'], sep=';')

data_test.head()

df = pd.read_csv('train_dataset_Самолет.csv', low_memory=False, parse_dates=['report_date'])

df.head()



data = df.dropna(thresh=int(len(df)*0.75), axis=1).copy()
data.drop(columns=['col1454', 'report_date'], inplace=True)

data_test = data_test.dropna(thresh=int(len(data_test)*0.75), axis=1).copy()


data_test

data = data.fillna(data.mean())

data_test.drop(columns='col1454', inplace=True)

data_test = data_test.fillna(data_test.mean())

data_test.head()

data.head()

data_f = data.copy()

client_id = data['client_id']

In [5]:
data_test.head()

,report_date,client_id,col1453,col1455,col1456,col1457,col1458,col1459,col1460,col1461,...,col2366,col2388,col2389,col2390,col2460,col2461,col2462,col2470,col2663,id
0,2023-04-01,2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,30.09,30.09,30.09,1.70,1.70,1.700000,299.0,0.256919,0
1,2023-05-01,4,0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,2.0,63.51,61.89,62.97,3.79,1.50,3.026667,31.0,0.267221,1
2,2023-05-01,9,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,49.92,49.92,49.92,1.34,1.34,1.340000,230.0,0.257217,2
3,2023-03-01,12,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,37.60,37.60,37.60,0.00,0.00,0.000000,391.0,0.256261,3
4,2023-03-01,14,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,37.05,37.05,37.05,2.20,2.20,2.200000,391.0,0.257348,4


In [6]:
data.head()

,client_id,target,col1453,col1455,col1456,col1457,col1458,col1459,col1460,col1461,...,col2365,col2366,col2388,col2389,col2390,col2460,col2461,col2462,col2470,col2663
0,1,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,37.88,37.88,37.880,0.00,0.00,0.00,545.000000,0.256261
1,5,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,38.75,3.34,21.045,0.00,0.00,0.00,242.000000,0.256261
2,6,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,36.75,36.75,36.750,2.10,2.10,2.10,557.000000,0.256261
3,7,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,35.56,35.56,35.560,1.40,1.40,1.40,332.556663,0.258682
4,8,0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,54.52,54.52,54.520,2.28,2.28,2.28,256.000000,0.254164


## Тестирование модели

In [7]:

features_test = data_test.drop(columns=['report_date'])
train = data.copy()

train.query('target == 1')

one_id = train.query('target == 1')['client_id']

one_id

criteria = 'client_id in @one_id'

train.loc[train.eval(criteria), 'target'] = 1

train.query('client_id in @one_id').loc[:, 'target']

features_train = train.drop(columns=['target', 'client_id'])

target_train = train['target']

features_train, target_train = SMOTE().fit_resample(features_train, target_train)

In [8]:
model = CatBoostClassifier(loss_function='Logloss',
    iterations=10000,
    custom_loss=['AUC', 'F1'],
    random_seed= RS,
    learning_rate= 0.2,
    verbose=False,
    random_strength =2,   
                           
    )

model.fit(features_train, target_train)




pred = model.predict(features_test)

In [9]:
preds_proba = model.predict_proba(features_test)

In [10]:
preds = pd.DataFrame(preds_proba).reset_index()

In [11]:
sub['target'] = preds[1]

In [12]:
sub

,id,target
0,0,0.000310
1,1,0.276901
2,2,0.000521
3,3,0.000007
4,4,0.000682
...,...,...
3637,3637,0.000013
3638,3638,0.000022
3639,3639,0.000009
3640,3640,0.000013


In [13]:
sub.to_csv('sub_14.csv', sep=';')